In [1]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, NonNegativeReals, Suffix
from pyomo.opt import SolverFactory
import pandas as pd

# 1. read all input data from files

relevant tsv-files are:
- load
- duration
- availability
- tech_data

In [33]:
load = pd.read_csv("load.tsv",  sep="\s+", names=["value"] )
load = load.T
load

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
value,82115,73169,68729,63442,60430,57013,52048,48701,43981,40498


In [34]:
tech_data = pd.read_csv("tech_data.tsv", sep="\s+", header=0, index_col=False, skiprows=[1], decimal=".") #workaround bc read csv shifts columnnames
tech_data.set_index("tech", inplace=True)
tech_data

,Cap,ETA_EL,Fuel_P,c_var_other,EMF
tech,,,,,
CCGT,29555,0.54,12.8,1.5,0.2048
GT_GasOil,4400,0.28,12.8,1.5,0.2048
Hydro,5256,1.00,0.0,1.5,0.0000
Coal,22458,0.42,7.4,2.6,0.3420
Lignite,21067,0.37,3.4,3.0,0.3996
Nuclear,8114,0.33,1.8,0.7,0.0000
Wind,61114,1.00,0.0,1.4,0.0000
Solar,46471,1.00,0.0,1.0,0.0000


In [35]:
availability = pd.read_csv("availability.tsv", sep="\s+")
availability

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
CCGT,19629,19629,19629,19629,19629,19629,19629,19629,19629,19629
GT_GasOil,2980,2980,2980,2980,2980,2980,2980,2980,2980,2980
Hydro,4012,4012,4012,4012,4012,4012,4012,4012,4012,4012
Coal,19564,19564,19564,19564,19564,19564,19564,19564,19564,19564
Lignite,17687,17687,17687,17687,17687,17687,17687,17687,17687,17687
Nuclear,7610,7610,7610,7610,7610,7610,7610,7610,7610,7610
Wind,11480,9858,11332,14498,20590,21012,6314,16072,21238,12054
Solar,10538,2078,3920,7641,6970,0,14810,1307,0,0


In [36]:
duration = pd.read_csv("duration.tsv", sep="\s+", names=["value"])
duration = duration.T
duration

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
value,102,962,962,962,962,962,962,962,962,962


# 2. set up calculations
if thats necessary.

In [37]:
tech_data["costs_el_no_co2"] = tech_data.Fuel_P / tech_data.ETA_EL + tech_data.c_var_other
tech_data.costs_el_no_co2

tech
CCGT         25.203704
GT_GasOil    47.214286
Hydro         1.500000
Coal         20.219048
Lignite      12.189189
Nuclear       6.154545
Wind          1.400000
Solar         1.000000
Name: costs_el_no_co2, dtype: float64

In [38]:
tech_data["emissions_el"] = tech_data.EMF / tech_data.ETA_EL
tech_data.emissions_el

tech
CCGT         0.379259
GT_GasOil    0.731429
Hydro        0.000000
Coal         0.814286
Lignite      1.080000
Nuclear      0.000000
Wind         0.000000
Solar        0.000000
Name: emissions_el, dtype: float64

In [39]:
co2_price = 50
tech_data["costs_el_w_co2"] = (tech_data.Fuel_P / tech_data.ETA_EL) + tech_data.c_var_other + (co2_price * tech_data.EMF / tech_data.ETA_EL)
tech_data

,Cap,ETA_EL,Fuel_P,c_var_other,EMF,costs_el_no_co2,emissions_el,costs_el_w_co2
tech,,,,,,,,
CCGT,29555,0.54,12.8,1.5,0.2048,25.203704,0.379259,44.166667
GT_GasOil,4400,0.28,12.8,1.5,0.2048,47.214286,0.731429,83.785714
Hydro,5256,1.00,0.0,1.5,0.0000,1.500000,0.000000,1.500000
Coal,22458,0.42,7.4,2.6,0.3420,20.219048,0.814286,60.933333
Lignite,21067,0.37,3.4,3.0,0.3996,12.189189,1.080000,66.189189
Nuclear,8114,0.33,1.8,0.7,0.0000,6.154545,0.000000,6.154545
Wind,61114,1.00,0.0,1.4,0.0000,1.400000,0.000000,1.400000
Solar,46471,1.00,0.0,1.0,0.0000,1.000000,0.000000,1.000000


# 3. configuration of model

model requires timesteps

In [50]:
model = ConcreteModel()

model.x = Var(tech_data.index.values, domain=NonNegativeReals) #dispatch variable

model.balance = Constraint(expr = sum(model.x[gen] for gen in tech_data.index.values) == load.t1.value) # meet demand

#TODO constraint availability

def cap_limits(model,gen):
    return model.x[gen] <= availability.loc[gen, "t1"]

model.cap_limits = Constraint(tech_data.index.values, rule=cap_limits)

model.objective = Objective(expr = sum([tech_data.costs_el_no_co2[gen]*model.x[gen] for gen in tech_data.index.values]))

In [51]:
opt = SolverFactory("glpk")

In [52]:
model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)

In [53]:
results = opt.solve(model,suffixes=["dual"])

In [54]:
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 973506.098087698
  Upper bound: 973506.098087698
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 9
  Number of nonzeros: 17
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.046889543533325195
# ----------------------------------------------------------
#   Solution Information
# ------------

In [55]:
for gen in tech_data.index.values:
    print(gen,model.x[gen].value)

CCGT 11224.0
GT_GasOil 0.0
Hydro 4012.0
Coal 19564.0
Lignite 17687.0
Nuclear 7610.0
Wind 11480.0
Solar 10538.0


In [56]:
model.dual[model.balance]

25.2037037037037